In [3]:
import pandas as pd
from lightgbm import LGBMClassifier

In [ ]:
clf = LGBMClassifier(
    objective="multiclass",
    n_estimators=10,
    num_class=9,
    seed=42,
    cat_feature=[0, 1, 2], # Нельзя передать список названий столбцов
)

clf.fit(
    X_train,
    y_train,
    eval_set=[
        (X_test, y_test),
    ],
)

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

# Пример данных (замените на свои)
# X, y = ...

# Разделение данных на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Оптимизация гиперпараметров
def objective(trial):
    # Гиперпараметры для DART
    param = {
        "boosting_type": "dart",
        "objective": "binary",
        "metric": "auc",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 500),
        "reg_alpha": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "drop_rate": trial.suggest_float("drop_rate", 0.1, 0.5),
        "skip_drop": trial.suggest_float("skip_drop", 0.1, 0.5),
        "random_state": 42
    }

    # Инициализация модели
    model = LGBMClassifier(**param)

    # Обучение модели
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="auc",
        early_stopping_rounds=50,
        verbose=False
    )

    # Предсказания и метрика
    preds = model.predict_proba(X_valid)[:, 1]  # Вероятности для положительного класса
    auc = roc_auc_score(y_valid, preds)
    return auc

# Создание и запуск Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Лучшие гиперпараметры
print("Best trial:")
print(study.best_trial.params)
